In [2]:
import os
import clip
import torch
import copy
import math
import json
from torch import nn
from tqdm import tqdm
from typing import Optional
import torch.nn.functional as F
from torch import Tensor
from torch.nested import as_nested_tensor
from torchvision.datasets import CocoDetection
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader

from model import build_model
from detr.models.detr import PostProcess, SetCriterion
from detr.models.matcher import build_matcher
import detr.util.misc as utils
from cocoeval import CocoEvaluator

In [3]:
with open('data/concepts_cleaned.json', 'r') as fp:
    concepts_dict = json.load(fp=fp)
concepts = []
for v in concepts_dict.values():
    concepts += v['gpt']
concepts = list(set(concepts))

In [10]:
[c.lower() for c in concepts]

['facial features',
 'hair',
 'body shape and size',
 'clothing',
 'unique distinguishing features',
 'two wheels',
 'frame',
 'handlebars',
 'pedals',
 'bicycle seat',
 'wheels',
 'license plate',
 'headlights',
 'rearview mirrors',
 'body shape',
 'two wheels',
 'handlebars',
 'engine',
 'exhaust pipe',
 'fairing or body panels',
 'wings',
 'tail fin',
 'cockpit',
 'landing gear',
 'propellers',
 'large size',
 'long and rectangular shape',
 'double-decker or single-decker structure',
 'windows along the sides',
 'bus number or signage displayed on the front or sides',
 'rectangular shape',
 'long and cylindrical body',
 'multiple carriages',
 'train tracks',
 'smokestack or locomotive at the front',
 'large size',
 'rectangular shape',
 'high ground clearance',
 'multiple wheels',
 'open cargo area',
 'shape',
 'floating',
 'hull',
 'mast',
 'rudder',
 'circular shape',
 'multiple colored lights',
 'vertical alignment of lights',
 'black housing/mounting pole',
 'light emission',
 '

In [7]:
len(concepts), len(set(concepts))

(400, 318)